In [79]:
#Plan
#json input parsing, extract snippet, comments
#prompt engineering: Ask for all features asked by the assignment, 
#Give full context(code, comment, purpose), 
#enforce format
# Output: positive paraphrasing, the Why, suggested improvement (snippet)
#add tooling to search for documentation links
# enforce tone change based on the severity of the review
# concluding paragraph to summarize feedback encouragingly
# few-shot examples for better results

# Creating a langchain agent along with prompt engineering

In [82]:
!pip install langchain-groq langchain-community duckduckgo-search

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  res = process_handler(cmd, _system_body)
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  res = process_handler(cmd, _system_body)
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  res = process_handler(cmd, _system_body)


In [83]:
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
import json
import re

### Load LLM

In [85]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=3000,
    api_key=""
)

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\pygments\regexopt.py:78: ResourceWarning: unclosed <ssl.SSLSocket fd=3076, family=23, type=1, proto=0, laddr=('2406:7400:43:9cab:d45f:450f:ed2d:2770', 55577, 0, 0), raddr=('2606:4700:4400::ac40:939e', 443, 0, 0)>
  for group in groupby(strings, lambda s: s[0] == first[0])) \


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

### Create Search Tool

In [ ]:
search_tool = DuckDuckGoSearchRun()

def create_documentation_search_tool():
    """Create tool for finding relevant documentation links"""
    
    def search_documentation(query: str) -> str:
        """Search for programming documentation and best practices"""
        # Enhanced search queries for better results
        enhanced_queries = [
            f"{query} official documentation",
            f"{query} best practices programming",
            f"{query} code style guide"
        ]
        
        results = []
        for enhanced_query in enhanced_queries[:2]:  # Limit for speed
            try:
                result = search_tool.run(enhanced_query)
                results.append(result)
            except:
                continue
                
        return "\n".join(results)
    
    return Tool(
        name="documentation_search",
        description="Search for relevant programming documentation, style guides, and best practices",
        func=search_documentation
    )

documentation_tool = create_documentation_search_tool()

### Prompts

In [ ]:
SEVERITY_CLASSIFICATION_PROMPT = """
Analyze the severity of these code review comments and classify each one:

CODE CONTEXT:
{code_snippet}

COMMENTS TO CLASSIFY:
{comments_list}

For each comment, classify as:
- CRITICAL: Security issues, major bugs, or performance problems
- MODERATE: Code quality, maintainability, or readability issues  
- MINOR: Style preferences, naming conventions, or minor optimizations

Return in format:
Comment 1: [SEVERITY] - [BRIEF_REASON]
Comment 2: [SEVERITY] - [BRIEF_REASON]
...
"""

In [ ]:
EMPATHETIC_REVIEW_PROMPT = """
You are a world-class senior software engineer and patient mentor known for transforming harsh criticism into constructive, encouraging guidance. Your mission is to help developers grow through empathetic feedback.

SEVERITY ANALYSIS:
{severity_analysis}

CODE SNIPPET:
{code_snippet}

ORIGINAL REVIEW COMMENTS:
{formatted_comments}

INSTRUCTIONS:
Transform each comment following this EXACT markdown format:

---
### Analysis of Comment: "[ORIGINAL_COMMENT]"

** Positive Rephrasing**: 
[Rewrite the comment with encouraging, supportive language. Adjust tone based on severity - more gentle for CRITICAL issues, friendly mentoring for MINOR issues]

** The 'Why'**: 
[Explain the underlying software engineering principle. Include analogies when helpful. Reference performance, readability, maintainability, or security as relevant]

** Suggested Improvement**:
[Provide concrete, working code example that addresses the issue. Include inline comments explaining the improvement. Use the documentation_search tool to find 1-2 relevant links to support the suggested improvement and add it to this field in bullet points with links]

---

TONE ADAPTATION RULES:
- CRITICAL issues: More formal, emphasize importance, focus on "let's fix this together"
- MODERATE issues: Balanced, educational, "here's a better approach"  
- MINOR issues: Casual, mentoring, "small improvement that will make your code shine"

After analyzing all comments, add:

## Overall Feedback Summary
[Write an encouraging 3-4 sentence summary highlighting the developer's strengths shown in the code, the learning opportunities from these improvements, and motivational next steps. Always end on a positive, growth-oriented note.]
"""


FEW_SHOT_EXAMPLES = """
EXAMPLE TRANSFORMATIONS:

Original: "This is inefficient. Don't loop twice conceptually."
Severity: MODERATE - Performance optimization opportunity

Positive Rephrasing: "Great logic flow here! I can see you're thinking through the filtering process step by step. For even better performance, especially with larger datasets, we can streamline this approach."

The 'Why': List iterations can become expensive with large datasets. Python's list comprehensions are not just more concise—they're actually optimized at the C level, making them faster for filtering operations like this.

Suggested Improvement:
def get_active_users(users):
# Single-pass filtering with list comprehension
return [user for user in users if user.is_active and user.profile_complete]

---

Original: "Variable 'u' is a bad name."
Severity: MINOR - Code readability improvement

Positive Rephrasing: "Nice work on the functionality! To make this code even more readable for your future self and teammates, let's use a more descriptive variable name."

The 'Why': Code is read far more often than it's written. Descriptive variable names act like documentation, making your intent crystal clear without needing comments.

Suggested Improvement:
More descriptive variable naming
for user in users:
if user.is_active and user.profile_complete:
results.append(user)

"""

In [98]:
class EmpathicCodeReviewer:
    def __init__(self):
        self.llm = llm
        self.documentation_tool = documentation_tool
        
        # Create agent with tool capability
        tools = [self.documentation_tool]
        tool_names = "documentation_search"
        react_prompt = ChatPromptTemplate.from_template("""
        You are an empathetic code reviewer who helps transform harsh feedback into supportive guidance.
        
        You have access to the following tools:
        {tools}
        
        Tool names: {tool_names}
        
        Use the following format:
        
        Question: the input question you must answer
        Thought: you should always think about what to do
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I have found the solution to what the user is asking in his question below in exact format
        Final Answer: Final answer made with all the instructions in the Question input below
        
        Begin!
        
        Question: {input}
        Thought: {agent_scratchpad}
        """)
        self.agent = create_react_agent(
            llm=self.llm,
            tools=tools,
            prompt=react_prompt
        )
        self.agent_executor = AgentExecutor(
            agent=self.agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=True
        )
        
    def classify_severity(self, code_snippet: str, comments: list) -> str:
        """Classify severity of each comment"""
        comments_text = "\n".join([f"{i+1}. {comment}" for i, comment in enumerate(comments)])
        
        prompt = ChatPromptTemplate.from_template(SEVERITY_CLASSIFICATION_PROMPT)
        
        response = self.llm.invoke(
            prompt.format(
                code_snippet=code_snippet,
                comments_list=comments_text
            )
        )
        return response.content
        
    def generate_empathetic_review(self, json_input: dict) -> str:
        """Main method to generate empathetic code review with tool integration"""
        
        # Extract data
        code_snippet = json_input["code_snippet"].replace('\\n', '\n')
        review_comments = json_input["review_comments"]
        
        # Classify severity
        severity_analysis = self.classify_severity(code_snippet, review_comments)
        
        # Format comments for prompt
        formatted_comments = "\n".join([f"{i+1}. {comment}" for i, comment in enumerate(review_comments)])
        
        # Create comprehensive prompt that instructs the agent to use tools
        empathetic_prompt = EMPATHETIC_REVIEW_PROMPT.format(
            severity_analysis=severity_analysis,
            code_snippet=code_snippet,
            formatted_comments=formatted_comments
        )
        
        # Use agent executor to generate response with tool access
        try:
            response = self.agent_executor.invoke({
                "input": empathetic_prompt
            })
            return response["output"]
        except Exception as e:
            # Fallback to direct LLM call without tools
            print(f"Agent execution failed: {e}")
            print("Falling back to direct LLM call...")
            
            response = self.llm.invoke(
                [HumanMessage(content=empathetic_prompt)]
            )
            return response.content

In [100]:
reviewer = EmpathicCodeReviewer()

In [102]:
example_input = {
  "code_snippet": "function calculateTotal(items) {\n  var sum = 0;\n  for (var i = 0; i < items.length; i++) {\n    sum += items[i].price * items[i].qty;\n  }\n  return sum;\n}",
  "review_comments": [
    "Use const/let instead of var. This is terrible practice.",
    "Function name is too generic.",
    "No input validation - this will crash with null items.",
    "Consider using reduce() for functional programming style."
  ]
}

In [104]:
result = reviewer.generate_empathetic_review(example_input)



> Entering new AgentExecutor chain...
To provide empathetic feedback on the given code snippet and comments, I need to analyze each comment based on its severity and transform it into a supportive guidance format.

Action: documentation_search
Action Input: "inefficient loop in python"

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


XNXX Today's selectionMi hermanastra pervertida sabe que no le puede decir a nuestros padres que follamos porque ya … XNXX delivers free sex movies and fast free porn videos (tube porn). Now 10 million+ sex vids available for free! Featuring hot … XNXX delivers free sex movies and fast free porn videos (tube porn). Now 10 million+ sex vids available for free! Featuring hot … XNXX.COM Sexy videos, free sex videosAll natural babe Gia Paige loves getting pussy stuffed, especially if it's about Jay … XNXX.COM Most Viewed Porn videos of the month, free sex videos
No good DuckDuckGo Search Result was foundThought: It seems like the documentation_search tool provided an irrelevant result. Let me try again with a more specific query.

Action: documentation_search
Action Input: "python list comprehension"

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


3 days ago · List comprehension is a way to create lists using a concise syntax. It allows us to generate a new list by applying an expression to each item in an existing iterable (such as a … Apr 5, 2025 · A well - documented Python list is crucial for both the developer who writes the code and others who may need to understand, maintain, or extend it. This blog will delve into the … Dec 20, 2024 · List Comprehension is one of the most powerful and expressive features of the Python programming language. It provides a concise and efficient way to create and process … Jun 2, 2025 · A list comprehension in Python is a way to create a new list based on the values of an existing lists using concise syntax. Here’s how to write them. Jun 8, 2025 · Learn Python list comprehensions with clear syntax, real examples, and common use cases. Master this Pythonic feature in minutes.
Jun 16, 2012 · There are two operators in Python for the "not equal" condition - a.) != If values of the two operands 

In [105]:
print(result)

### Analysis of Comment: "This is inefficient. Don't loop twice conceptually."
**Positive Rephrasing**: 
Let's explore a more efficient approach to achieve the same result. We can simplify the code and reduce the number of iterations.
**The 'Why'**: 
The original code uses a for loop to iterate over the users, which is a good start. However, the condition `u.is_active == True and u.profile_complete == True` can be simplified. In Python, we can directly use the boolean values of `u.is_active` and `u.profile_complete` without comparing them to `True`. This simplification can improve readability and maintainability.
**Suggested Improvement**:
```python
def get_active_users(users):
    # Use list comprehension to create a new list with active and complete users
    results = [u for u in users if u.is_active and u.profile_complete]
    return results
```
* [Python List Comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp)
* [Python Boolean Values](https://www.geeks